# Importando as bibliotecas

In [ ]:
import pandas as pd
from db import connection_db as conndb
from db import filters
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from utils.estracao_interacao import ExtracaoInteracao
from utils.text_treatment import TextTreatment
from utils.busca_palavras import BuscaPalavras

tqdm.pandas()

Conectando ao banco de dados do MongoDB

In [ ]:
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
col_name = 'dadosSemFiltros'
col_filtrada = 'posts7anos1anos'

mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=col_name)
mongo_connection.connect()
collection = mongo_connection.collection

mongo_connection_filtrada = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=col_filtrada)
mongo_connection_filtrada.connect()
collection_filtrada = mongo_connection_filtrada.collection

Aplicando pipeline para preparação dos dados

In [ ]:
collection_filters = filters.CollectionFilters(collection)

collection_filters_likes = filters.CollectionFilters(collection_filtrada)

# Aplicando pipeline 1
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')

# Aplicando pipeline 2 - pega os posts que tenha a data de publicação de no máximo 7 anos e no mínimo 2 anos
collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')

# Aplicando pipeline 3 - desenrola os posts em documentos individuais e cria novas colunas
collection_filters.apply_pipeline3('postsComBDIAndInfos')

# Aplicando pipeline 4 - aplica o filtro para selecionar apenas os posts que tem uma data de publicação válida
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')

collectionLikes = collection_filters_likes.apply_pipeline6('likes')

collection = collection_filters.collection

In [ ]:
documentos = collection.find()
df_original = pd.DataFrame(list(documentos))

# Visualizar as primeiras linhas do dataframe
df_original.head()

In [ ]:
documentos_likes = collectionLikes.find()
df_likes = pd.DataFrame(list(documentos_likes))

In [ ]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]


# Função para preencher valores nulos e converter o tipo de dado
def preencher_e_converter(df, colunas, valor_preenchimento=0, tipo_dados='int64'):
    df[colunas] = df[colunas].fillna(valor_preenchimento)
    df[colunas] = df[colunas].astype(tipo_dados)
    return df


# Aplicando a função
df_original = preencher_e_converter(df_original, colunas_para_converter)

# Filtragem dos dados
df_original.drop(columns=['_id', 'diaDaSemana', 'hora', 'minutos', 'diaDoMes', 'mes', 'ano'], inplace=True)

# Criando novas colunas com o pandas
df_original['data'] = df_original['postCreatedTime'].dt.date
df_original['data'] = pd.to_datetime(df_original['data'])
df_original['mes'] = df_original['data'].dt.to_period('M')
df_original['semana'] = df_original['data'].dt.to_period('W')

# Filtrar posts do ano de 2016
df_2017 = df_original[df_original['data'].dt.year == 2017]
df_2017 = df_2017[~((df_2017['data'].dt.month == 12) & (df_2017['data'].dt.year == 2017))]
df_2017 = df_2017[df_2017['data'].dt.month >= 5]
df_2017 = df_2017[df_2017['suicida'] == 3]
df_2017 = df_2017

# Remover linhas onde ambas as colunas 'postMessage' e 'postStory' estão vazias ou nulas
df_2017 = df_2017.dropna(subset=['postMessage', 'postStory'], how='all')  # Remove quando ambas são NaN
df_2017 = df_2017[~((df_2017['postMessage'].str.strip() == '') & (
        df_2017['postStory'].str.strip() == ''))]  # Remove quando ambas são strings vazias

In [ ]:
# Adicionando novas features
extracao = ExtracaoInteracao(df_2017)
df_interacoes = extracao.extract_interactions()

# Passo 6: Modificar outras colunas usando .loc para evitar avisos
df_interacoes['sexo'] = df_interacoes['sexo'].map({'F': 0, 'M': 1})

tratamento_texto = TextTreatment()
df_interacoes['postMessageLimpo'] = df_interacoes['postMessage'].fillna('').progress_apply(
    lambda texto: tratamento_texto.preprocessamento_texto(texto) if texto else '')

busca_palavras = BuscaPalavras()

# Faz a busca exata
resultado = busca_palavras.string_matching(df_interacoes['postMessageLimpo'],
                                           'dados/datasets/termos_depressivos_pt_br.txt')

# Adicionar a coluna quantPalavrasDepressivas ao DataFrame, contando quantas palavras depressivas foram encontradas
df_interacoes['quantPalavrasDepressivas'] = resultado.apply(lambda x: len(x.split(', ')) if x else 0)
df_interacoes.head()

In [ ]:
# Criar um dataframe para conter as colunas que não mudam e o id_usuario
df_respostas_bdi = df_interacoes.groupby(['id_usuario']).agg(
    idade=('idade', 'first'),
    sexo=('sexo', 'first'),
    quantAmigos=('quantAmigos', 'first'),
    nivel=('nivel', 'first'),
    pessimismo=('pessimismo', 'first'),
    tristeza=('tristeza', 'first'),
    fracasso=('fracasso', 'first'),
    prazer=('prazer', 'first'),
    culpa=('culpa', 'first'),
    punicao=('punicao', 'first'),
    estima=('estima', 'first'),
    critica=('critica', 'first'),
    suicida=('suicida', 'first'),
    choro=('choro', 'first'),
    agitacao=('agitacao', 'first'),
    interesse=('interesse', 'first'),
    indecisao=('indecisao', 'first'),
    desvalorizacao=('desvalorizacao', 'first'),
    energia=('energia', 'first'),
    sono=('sono', 'first'),
    irritabilidade=('irritabilidade', 'first'),
    apetite=('apetite', 'first'),
    concentracao=('concentracao', 'first'),
    fadiga=('fadiga', 'first'),
    int_sexo=('int_sexo', 'first'),
).reset_index()

# Visualizar as primeiras linhas do dataframe
df_respostas_bdi.head()

In [ ]:
df_interacoes.drop(
    columns=['idade', 'sexo', 'quantAmigos', 'nivel', 'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa',
             'punicao', 'estima', 'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao', 'desvalorizacao',
             'energia', 'sono', 'irritabilidade', 'apetite', 'concentracao', 'fadiga', 'int_sexo'], inplace=True)

In [ ]:
df_interacoes.head()

In [ ]:
# Adicionar a coluna quantPostMessage: contar onde postMessage não é vazio ou nulo
df_interacoes['quantPostMessage'] = df_interacoes['postMessage'].apply(
    lambda x: 1 if pd.notnull(x) and x.strip() != '' else 0)

In [ ]:
df_interacoes.head()

In [ ]:
# Criar novos dataframe, df_poucas_interacoes, df_normal e df_muitas_interacoes
# Lista das variáveis que variam com o tempo
variaveis_interacao = [
    'quantPostMessage', 'quantProfile', 'quantCover', 'quantAddPhotoWithOthers', 'quantIsWithOthers', 'quantAddPhoto',
    'quantSharedPhoto', 'quantSharedVideo', 'quantSharedLink', 'quantSharedPost', 'quantSharedEvent',
    'quantSharedMemory', 'quantStatus', 'quantPalavrasDepressivas'
]